In [1]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import itertools
import time
from datetime import date, timedelta
import datetime
from tqdm import tqdm
from interruptingcow import timeout
from interruptingcow import timeout
import re
import string
punctuation_string = string.punctuation
tqdm.pandas()
import gc

Top 100 products from dataset
plus news channels
plus Martin Lewis
What else?

In [2]:
df = pd.read_csv('Datasetjson.csv', low_memory=False)
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290029 entries, 0 to 290028
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         290029 non-null  float64
 1   vote            48045 non-null   object 
 2   verified        290029 non-null  bool   
 3   reviewTime      290029 non-null  object 
 4   reviewerID      290029 non-null  object 
 5   asin            290029 non-null  object 
 6   reviewerName    290006 non-null  object 
 7   reviewText      289896 non-null  object 
 8   summary         289960 non-null  object 
 9   unixReviewTime  290029 non-null  int64  
 10  style           171725 non-null  object 
 11  image           2705 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 24.6+ MB


In [3]:
df_meta= pd.read_csv('meta.csv', low_memory=False)
df_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14751104 entries, 0 to 14751103
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   brand     object
 1   asin      object
 2   category  object
dtypes: object(3)
memory usage: 337.6+ MB


In [4]:
df['review'] = df['reviewText']+' '+df['summary']
df['review'].head()

0    I bought two of these 8.5 fl oz hand cream, an...
1    Believe me, over the years I have tried many, ...
2                         Great hand lotion Five Stars
3    This is the best for the severely dry skin on ...
4    The best non- oily hand cream ever. It heals o...
Name: review, dtype: object

In [5]:
df.groupby('asin').count()
df[df['asin']=='0209688726']

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,review
10001,4.0,3,True,"08 12, 2015",A3ELV1VUCZZGL4,0209688726,AnaG,it's fine. I just would like the stickers to b...,Serve its purpose,1439337600,{'Color:': ' CA'},['https://images-na.ssl-images-amazon.com/imag...,it's fine. I just would like the stickers to b...
10002,2.0,NaN,True,"08 12, 2015",A2FD3FZ2WV6KHQ,0209688726,Obi John Kenobi,took me three returns to get one that didn't w...,and it's still not super stable but whatever,1439337600,{'Color:': ' T'},NaN,took me three returns to get one that didn't w...
10003,2.0,12,True,"08 11, 2015",A39NRKABAVSVSE,0209688726,MuddyRiverGirl,While the product is fine the description and ...,Beware! Description is wrong.,1439251200,{'Color:': ' AC'},['https://images-na.ssl-images-amazon.com/imag...,While the product is fine the description and ...
10004,5.0,NaN,True,"07 29, 2015",AQSBL5KODHSGW,0209688726,Nate S,It's beautiful and blends right in with my woo...,"Awesome desk stand, simple and elegant",1438128000,{'Color:': ' T'},NaN,It's beautiful and blends right in with my woo...
10005,5.0,NaN,False,"07 11, 2015",AU45OQPK2J5JF,0209688726,Kort,I love this stand! I had been looking around f...,I love this stand,1436572800,{'Color:': ' T'},NaN,I love this stand! I had been looking around f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11242,5.0,NaN,False,"09 10, 2015",A3EGBGSVUD7PVH,0209688726,Momma gamma,I purchased two different ultrasonic diffusers...,excellent first diffuser,1441843200,{'Color:': ' J'},NaN,I purchased two different ultrasonic diffusers...
11243,5.0,NaN,True,"09 1, 2015",A8UYC4W0H6MP0,0209688726,Faaiz Khan,Everyone notices this on my desk! I am even co...,love it. simple and sleek,1441065600,{'Color:': ' T'},NaN,Everyone notices this on my desk! I am even co...
11244,3.0,NaN,True,"08 25, 2015",A3NNZTYM2TVNN8,0209688726,Kindle Customer,Holds 1-3 keys great but hard to get one set o...,Holds 1-3 keys great but hard to get one set o...,1440460800,{'Color:': ' CA'},NaN,Holds 1-3 keys great but hard to get one set o...
11245,5.0,4,True,"08 22, 2015",A3NMHKHMUNFB33,0209688726,Stacy King,I love how compact and easy to move rooms this...,Petite & Powerful,1440201600,{'Color:': ' I'},NaN,I love how compact and easy to move rooms this...


In [6]:
df_meta = df_meta.drop_duplicates(subset=['asin', 'brand', 'category'])
df_meta

,brand,asin,category
0,BubuBibi,0000032042,"['Sports & Outdoors', 'Sports & Fitness', 'Oth..."
1,BubuBibi,0000032069,"['Sports & Outdoors', 'Sports & Fitness', 'Oth..."
2,Unknown,0000031860,"['Sports & Outdoors', 'Sports & Fitness', 'Oth..."
3,Tutu,0000031852,"['Sports & Outdoors', 'Sports & Fitness', 'Oth..."
4,Bububibi,0000031895,"['Sports & Outdoors', 'Sports & Fitness', 'Oth..."
...,...,...,...
14751099,NaN,B01GMFJ8JU,"['Gift Cards', 'Gift Cards']"
14751100,QuickGifts,B01GOQIKRI,"['Gift Cards', 'Gift Cards']"
14751101,QuickGifts,B01GOQHGL4,"['Gift Cards', 'Gift Cards']"
14751102,Go Play Golf by Fairway Rewards,B01GP1W4LA,"['Gift Cards', 'Gift Cards']"


In [7]:
df_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14064249 entries, 0 to 14751103
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   brand     object
 1   asin      object
 2   category  object
dtypes: object(3)
memory usage: 429.2+ MB


In [8]:
df_merge = df.merge(df_meta, on='asin')

In [9]:
df_merge

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,review,brand,category
0,2.0,3,True,"06 15, 2010",A1Q6MUU0B2ZDQG,B00004U9V2,D. Poston,"I bought two of these 8.5 fl oz hand cream, an...",dispensers don't work,1276560000,NaN,NaN,"I bought two of these 8.5 fl oz hand cream, an...",NaN,[]
1,5.0,14,True,"01 7, 2010",A3HO2SQDCZIE9S,B00004U9V2,chandra,"Believe me, over the years I have tried many, ...",Best hand cream ever.,1262822400,NaN,NaN,"Believe me, over the years I have tried many, ...",NaN,[]
2,5.0,NaN,True,"04 18, 2018",A2EM03F99X3RJZ,B00004U9V2,Maureen G,Great hand lotion,Five Stars,1524009600,{'Size:': ' 3.5 oz.'},NaN,Great hand lotion Five Stars,NaN,[]
3,5.0,NaN,True,"04 18, 2018",A3Z74TDRGD0HU,B00004U9V2,Terry K,This is the best for the severely dry skin on ...,Five Stars,1524009600,{'Size:': ' 3.5 oz.'},NaN,This is the best for the severely dry skin on ...,NaN,[]
4,5.0,NaN,True,"04 17, 2018",A2UXFNW9RTL4VM,B00004U9V2,Patricia Wood,The best non- oily hand cream ever. It heals o...,I always have a backup ready.,1523923200,{'Size:': ' 3.5 oz.'},NaN,The best non- oily hand cream ever. It heals o...,NaN,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263011,4.0,NaN,True,"09 11, 2012",AEC1OF404YH8Z,B000JML31E,momoftwo,"After receiving a Kindle for Christmas, I want...",Better than the movies,1347321600,{'Format:': ' Kindle Edition'},NaN,"After receiving a Kindle for Christmas, I want...",Daniel Defoe,"['Kindle Store', 'Kindle eBooks', 'History']"
263012,5.0,NaN,True,"09 6, 2012",A35XTCFNWGZPUK,B000JML31E,Kindle Customer,One of the most well writen stories I have rea...,robinson crusoe,1346889600,{'Format:': ' Kindle Edition'},NaN,One of the most well writen stories I have rea...,Daniel Defoe,"['Kindle Store', 'Kindle eBooks', 'History']"
263013,5.0,4,True,"03 11, 2012",ADGIVL5ZTEEZB,B000JML31E,Robert J. Clancy,I read this account when I was in high school ...,The Original & Still Best Desert Island/Shipwr...,1331424000,{'Format:': ' Kindle Edition'},NaN,I read this account when I was in high school ...,Daniel Defoe,"['Kindle Store', 'Kindle eBooks', 'History']"
263014,5.0,10,True,"02 24, 2012",AZWOQXRCS1WA6,B000JML31E,D. D. LeDu,I read a child's version of this book when onl...,Must be read again!,1330041600,{'Format:': ' Kindle Edition'},NaN,I read a child's version of this book when onl...,Daniel Defoe,"['Kindle Store', 'Kindle eBooks', 'History']"


In [10]:
df_merge[df_merge.brand.notnull()].groupby('brand').count().sort_values(by='reviewText', ascending=False)

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,review,category
brand,,,,,,,,,,,,,,
Black Mountain,7427,458,7427,7427,7427,7427,7427,7426,7423,7427,661,99,7422,7427
nobrand,5299,617,5299,5299,5299,5299,5299,5295,5297,5299,4236,83,5294,5299
Hosa,4653,210,4653,4653,4653,4653,4653,4653,4653,4653,4559,24,4653,4653
Days of Wonder,4159,248,4159,4159,4159,4159,4159,4156,4159,4159,0,31,4156,4159
Mighty Bright,3963,419,3963,3963,3963,3963,3962,3963,3961,3963,3114,19,3961,3963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ludwig van Beethoven,1,0,1,1,1,1,1,1,1,1,0,0,1,1
Love Song,1,1,1,1,1,1,1,1,1,1,1,0,1,1
Louis Armstrong,1,1,1,1,1,1,1,1,1,1,1,0,1,1


In [11]:
df_merge.dropna().reset_index(drop=True)

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,review,brand,category
0,1.0,4,True,"10 1, 2016",A3MMO1UVNM7D0H,B00002243Z,T. Andrew,Very poorly made in China. Look at the pictur...,Garbage. Look elsewhere,1475280000,{'Size:': ' 25-Feet'},['https://images-na.ssl-images-amazon.com/imag...,Very poorly made in China. Look at the pictur...,Coleman Cable,"['Automotive', 'Replacement Parts', 'Batteries..."
1,3.0,131,True,"03 28, 2010",AWKYTPVNNZELL,B00002243Z,T. Brown,For the price and in the right situations thes...,"Good Jumper Cables for the money, But.....",1269734400,{'Size:': ' 20-Feet'},['https://images-na.ssl-images-amazon.com/imag...,For the price and in the right situations thes...,Coleman Cable,"['Automotive', 'Replacement Parts', 'Batteries..."
2,3.0,5,True,"09 17, 2016",A242UHNRH3GNQK,B00004T81V,welder314,This tool is a usable option if you do occasio...,"Nice concept, poor execution",1474070400,{'Style:': ' Plier Type'},['https://images-na.ssl-images-amazon.com/imag...,This tool is a usable option if you do occasio...,General Tools,"['Automotive', 'Tools & Equipment', 'Brake Rep..."
3,3.0,13,True,"12 5, 2015",AO53T9NBLHN1Q,B00004T81V,Juan S.,It really didn't help that no one might of tok...,Not really what I expected,1449273600,{'Style:': ' Plier Type'},['https://images-na.ssl-images-amazon.com/imag...,It really didn't help that no one might of tok...,General Tools,"['Automotive', 'Tools & Equipment', 'Brake Rep..."
4,2.0,8,True,"05 14, 2016",A5VCQ0V8WDDCQ,B000056OQL,big_art,I was excited to try this on my new Porter Cab...,Foam Pad Separated From Disc - First Use,1463184000,{'Package Quantity:': ' 1'},['https://images-na.ssl-images-amazon.com/imag...,I was excited to try this on my new Porter Cab...,PORTER-CABLE,"['Automotive', 'Tools & Equipment', 'Body Repa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,1.0,12,True,"12 6, 2015",A2T91JAAI4L9TU,7508229568,Dennis Shackleford,I purchased a 10 pack of these bulbs in 2014. ...,THESE BULBS EXPLODE AND ARE A SAFTY HAZARD - D...,1449360000,"{'Size:': ' 5 Pack', 'Color:': ' 1.7 Watts, Wa...",['https://images-na.ssl-images-amazon.com/imag...,I purchased a 10 pack of these bulbs in 2014. ...,CBconcept,"['Tools & Home Improvement', 'Light Bulbs', 'H..."
760,1.0,3,True,"12 18, 2015",A2TY2TJU73NTUH,7504336831,G. Smith,I ordered several LED ropes from CBconcept thr...,I strongly recommend staying with recognized b...,1450396800,{'Color:': ' Warm White (3000K)'},['https://images-na.ssl-images-amazon.com/imag...,I ordered several LED ropes from CBconcept thr...,CBconcept,"['Tools & Home Improvement', 'Lighting & Ceili..."
761,5.0,11,True,"02 17, 2015",AL7Y0MVLJYT9C,750823765X,evan2293,Worked great in my bathroom light and for the ...,Worked great in my bathroom light and for the ...,1424131200,{'Size:': ' 60 Watt'},['https://images-na.ssl-images-amazon.com/imag...,Worked great in my bathroom light and for the ...,CBconcept,"['Tools & Home Improvement', 'Light Bulbs', 'H..."
762,5.0,2,True,"09 27, 2016",A3TR7C81RIXT5Q,7532434621,Robert Jones,This bulb works great in my low voltage mini s...,Worked great in my mini low voltage spotlight,1474934400,"{'Size:': ' 2 Pack', 'Color:': ' Warm White 30...",['https://images-na.ssl-images-amazon.com/imag...,This bulb works great in my low voltage mini s...,CBconcept,"['Tools & Home Improvement', 'Light Bulbs', 'H..."


In [12]:
df_merge[df_merge['brand'].notnull()]

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,review,brand,category
10001,5.0,NaN,True,"03 20, 2015",A3M20IGF19WMBD,7138258879,julie,The were good,Five Stars,1426809600,NaN,NaN,The were good Five Stars,Elite Mailers,"['Automotive', 'Exterior Accessories', 'Bumper..."
10002,3.0,NaN,True,"01 14, 2015",A2GGCRLQ75WJFE,7138258879,Subiechick,One sticker was incomplete and missing letters...,Incomplete,1421193600,NaN,NaN,One sticker was incomplete and missing letters...,Elite Mailers,"['Automotive', 'Exterior Accessories', 'Bumper..."
10003,5.0,NaN,True,"01 3, 2015",A2M7MSLDRV6FC6,7138258879,Cris Canoneo,Thank you,Five Stars,1420243200,NaN,NaN,Thank you Five Stars,Elite Mailers,"['Automotive', 'Exterior Accessories', 'Bumper..."
10004,5.0,NaN,False,"11 1, 2014",A3RKXT2JD6KZR0,7138258879,Mr. Sam,Great space filler that will impress a lot of ...,Great space filler that will impress a lot of ...,1414800000,NaN,NaN,Great space filler that will impress a lot of ...,Elite Mailers,"['Automotive', 'Exterior Accessories', 'Bumper..."
10005,5.0,NaN,False,"09 14, 2014",A2IJ5951R8OXLT,7138258879,Silveradolife04,Great product sticks well gone through several...,Fantastic,1410652800,NaN,['https://images-na.ssl-images-amazon.com/imag...,Great product sticks well gone through several...,Elite Mailers,"['Automotive', 'Exterior Accessories', 'Bumper..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263011,4.0,NaN,True,"09 11, 2012",AEC1OF404YH8Z,B000JML31E,momoftwo,"After receiving a Kindle for Christmas, I want...",Better than the movies,1347321600,{'Format:': ' Kindle Edition'},NaN,"After receiving a Kindle for Christmas, I want...",Daniel Defoe,"['Kindle Store', 'Kindle eBooks', 'History']"
263012,5.0,NaN,True,"09 6, 2012",A35XTCFNWGZPUK,B000JML31E,Kindle Customer,One of the most well writen stories I have rea...,robinson crusoe,1346889600,{'Format:': ' Kindle Edition'},NaN,One of the most well writen stories I have rea...,Daniel Defoe,"['Kindle Store', 'Kindle eBooks', 'History']"
263013,5.0,4,True,"03 11, 2012",ADGIVL5ZTEEZB,B000JML31E,Robert J. Clancy,I read this account when I was in high school ...,The Original & Still Best Desert Island/Shipwr...,1331424000,{'Format:': ' Kindle Edition'},NaN,I read this account when I was in high school ...,Daniel Defoe,"['Kindle Store', 'Kindle eBooks', 'History']"
263014,5.0,10,True,"02 24, 2012",AZWOQXRCS1WA6,B000JML31E,D. D. LeDu,I read a child's version of this book when onl...,Must be read again!,1330041600,{'Format:': ' Kindle Edition'},NaN,I read a child's version of this book when onl...,Daniel Defoe,"['Kindle Store', 'Kindle eBooks', 'History']"


In [13]:
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuation_string)
    return text.translate(translator)

#df_merge = df_merge.dropna().reset_index(drop=True)
df_merge = df_merge[df_merge['brand'].notnull()]
print('cleaning punctuations')
clean_punct = df_merge['brand'].progress_apply(lambda x: cleaning_punctuations(x))
df_merge = df_merge.assign(brand=clean_punct)

cleaning punctuations


100%|██████████| 236592/236592 [00:02<00:00, 87076.39it/s] 


In [14]:
remove_visit = df_merge['brand'].progress_apply(lambda x: re.sub('Visit Amazons ','', x))
df_merge= df_merge.assign(brand=remove_visit)

100%|██████████| 236592/236592 [00:01<00:00, 161432.89it/s]


In [15]:
remove_amazonpage = df_merge['brand'].progress_apply(lambda x: re.sub(' Page','', x))

100%|██████████| 236592/236592 [00:01<00:00, 167974.65it/s]


In [16]:
df_merge= df_merge.assign(brand=remove_amazonpage)

In [17]:
df_merge.groupby('brand').count()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,review,category
brand,,,,,,,,,,,,,,
101 Strings Orchestra,56,12,56,56,56,56,56,56,56,56,56,0,56,56
10cc,1,1,1,1,1,1,1,1,1,1,1,0,1,1
13Rainbow,9,2,9,9,9,9,9,9,9,9,0,0,9,9
2nd Chapter of Acts,18,6,18,18,18,18,18,18,18,18,11,0,18,18
2ne1,7,1,7,7,7,7,7,7,7,7,7,0,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nobrand,5299,617,5299,5299,5299,5299,5299,5295,5297,5299,4236,83,5294,5299
sela,1,1,1,1,1,1,1,1,1,1,0,0,1,1
sm,4,1,4,4,4,4,4,4,4,4,0,0,4,4


In [18]:
text = 'Visit Amazons Zondervan Page'
re.search('Visit Amazons [a-zA-Z]+ Page', text)

<re.Match object; span=(0, 28), match='Visit Amazons Zondervan Page'>

In [19]:
remove_by = df_merge['brand'].progress_apply(lambda x: re.sub('by\n    \n    ','' ,x))

100%|██████████| 236592/236592 [00:01<00:00, 150522.60it/s]


In [20]:
df_merge= df_merge.assign(brand=remove_by)
#df_merge.loc[18624]

In [21]:
df_products = df_merge.groupby('brand').count().sort_values('overall', ascending=False)

In [22]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1654 entries, Black Mountain to Lin Carter
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   overall         1654 non-null   int64
 1   vote            1654 non-null   int64
 2   verified        1654 non-null   int64
 3   reviewTime      1654 non-null   int64
 4   reviewerID      1654 non-null   int64
 5   asin            1654 non-null   int64
 6   reviewerName    1654 non-null   int64
 7   reviewText      1654 non-null   int64
 8   summary         1654 non-null   int64
 9   unixReviewTime  1654 non-null   int64
 10  style           1654 non-null   int64
 11  image           1654 non-null   int64
 12  review          1654 non-null   int64
 13  category        1654 non-null   int64
dtypes: int64(14)
memory usage: 193.8+ KB


In [23]:
df_products = df_products.rename_axis('brand').reset_index()


In [24]:
top_100 = df_products['brand'].head(100)
top_100_list = top_100.to_list()

#top_100_list.remove('Unknown')
top_100_list.remove('nobrand')

In [25]:
top_100_list

['Black Mountain',
 'Hosa',
 'Nintendo',
 'Days of Wonder',
 'Mighty Bright',
 'CBconcept',
 'Gilmour',
 'Microsoft',
 'Scholastic',
 'Fiskars',
 'Panasonic',
 'The Elf on the Shelf',
 'Barnes amp Noble',
 'Wilton',
 'PORTERCABLE',
 'Multipet',
 'Dr Seuss',
 'XActo',
 'Davids Cookies',
 'Ava Miles',
 'Holmes',
 'Bonnier Corporation',
 'Braun',
 'Carnation',
 'Dr Harvey Karp',
 'Hearst Magazines',
 'LitterMaid',
 'Trusted Media Brands Inc',
 'Sally Field',
 'Karl Marx',
 'Progresso',
 'Nesco American Harvest',
 'Werner',
 'CS Lewis',
 'VideoSecu',
 'Zondervan',
 'Conde Nast Publications',
 'Dremel',
 'SET Enterprises',
 'Presto',
 'PowerBear',
 'Yamaha',
 'Motorola',
 'Nelson',
 'Reston Lloyd',
 'Winsor amp Newton',
 'BestAir',
 'California Car Duster',
 'Samsung',
 'Makita',
 'AMERICAN CREW',
 'The Learning Company',
 'Dole',
 'Wendy Crewson',
 'DRREXY',
 'Playskool',
 'Metallica',
 'Frances Hodgson illustrated by ruth sanderson Burnett',
 'Bayer Animal Health',
 'ernest hemingway',
 '

In [26]:
df_tweets = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang'])
start_date = date(2015, 12, 31)
end_date = date(2022, 8, 2)

df_collect = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
    f'{top_100[0]}  since:{start_date} until:{end_date} within: 10km' ).get_items(), 100))
            #[['date', 'content', 'user','hashtags', 'coordinates', 'lang']]

In [27]:
df_collect = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
    f'geocode:53.81653,-3.03033,320km since:2022-07-01 until:2022-07-31').get_items(), 100))

In [28]:
df_collect

,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,...,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags
0,https://twitter.com/Imranicus/status/155353087...,2022-07-30 23:59:57+00:00,#Tories looking at the state of the economy: h...,#Tories looking at the state of the economy: h...,1553530870732132353,"{'username': 'Imranicus', 'id': 11863826114317...",0,0,2,0,...,[{'previewUrl': 'https://pbs.twimg.com/media/F...,None,None,NaN,None,None,"{'longitude': -2.659936, 'latitude': 51.399367}","{'fullName': 'Bristol, England', 'name': 'Bris...",[Tories],None
1,https://twitter.com/DavidJCourt/status/1553530...,2022-07-30 23:59:55+00:00,@TAGartwork I had a frustrating one.\n\n🟩🟩🟩⬜⬜\...,@TAGartwork I had a frustrating one.\n\n🟩🟩🟩⬜⬜\...,1553530864637808641,"{'username': 'DavidJCourt', 'id': 6642372, 'di...",1,0,0,0,...,None,None,None,1.553519e+18,"{'username': 'TAGartwork', 'id': 221710311, 'd...","[{'username': 'TAGartwork', 'id': 221710311, '...","{'longitude': -1.6128929, 'latitude': 52.3653979}","{'fullName': 'Coventry, England', 'name': 'Cov...",None,None
2,https://twitter.com/JKJeonEuphoria/status/1553...,2022-07-30 23:59:50+00:00,They're obv gonna do this right? Meaning Chica...,They're obv gonna do this right? Meaning Chica...,1553530842902822913,"{'username': 'JKJeonEuphoria', 'id': 123198962...",0,0,1,0,...,None,None,None,NaN,None,None,"{'longitude': -2.0971002, 'latitude': 52.460709}","{'fullName': 'West Bromwich, England', 'name':...",None,None
3,https://twitter.com/Mofoman360/status/15535308...,2022-07-30 23:59:48+00:00,Come on dudes who is looking forward to next w...,Come on dudes who is looking forward to next w...,1553530833381859328,"{'username': 'Mofoman360', 'id': 20820445, 'di...",1,1,5,0,...,None,None,None,NaN,None,None,"{'longitude': -1.706057, 'latitude': 53.7363694}","{'fullName': 'Leeds, England', 'name': 'Leeds'...","[mot, alaw, lufc, LeedsUnited]",None
4,https://twitter.com/sophielordx1/status/155353...,2022-07-30 23:59:47+00:00,Right. Ive bought a paddle board,Right. Ive bought a paddle board,1553530829799923712,"{'username': 'sophielordx1', 'id': 576408734, ...",0,0,0,0,...,None,None,None,NaN,None,None,"{'longitude': -2.5139084, 'latitude': 53.531967}","{'fullName': 'Bolton, England', 'name': 'Bolto...",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,https://twitter.com/TheBeautyNurse2/status/155...,2022-07-30 23:54:37+00:00,@loyalEllis2 Because her family reached out to...,@loyalEllis2 Because her family reached out to...,1553529528043159557,"{'username': 'TheBeautyNurse2', 'id': 14440558...",0,0,3,0,...,None,None,None,1.553392e+18,"{'username': 'loyalEllis2', 'id': 143322503288...","[{'username': 'loyalEllis2', 'id': 14332250328...","{'longitude': -2.033651, 'latitude': 52.381063}","{'fullName': 'Birmingham, England', 'name': 'B...",None,None
96,https://twitter.com/JKJeonEuphoria/status/1553...,2022-07-30 23:54:35+00:00,I know this isn't a typical festival song but ...,I know this isn't a typical festival song but ...,1553529520053002241,"{'username': 'JKJeonEuphoria', 'id': 123198962...",2,2,2,2,...,None,None,None,NaN,None,None,"{'longitude': -2.0971002, 'latitude': 52.460709}","{'fullName': 'West Bromwich, England', 'name':...",None,None
97,https://twitter.com/Luiizchagas/status/1553529...,2022-07-30 23:54:34+00:00,"@jumartinellii Eu ia vê, mas preferi assistir ...","@jumartinellii Eu ia vê, mas preferi assistir ...",1553529515317645319,"{'username': 'Luiizchagas', 'id': 187282748, '...",1,0,0,0,...,None,None,None,1.553457e+18,"{'username': 'jumartinellii', 'id': 7278511516...","[{'username': 'jumartinellii', 'id': 727851151...","{'longitude': -2.319934, 'latitude': 53.343623}","{'fullName': 'Manchester, England', 'name': 'M...",None,None
98,https://twitter.com/susannahmh/status/15535294...,2022-07-30 23:54:30+00:00,@BarryHavenhand A shorter word. E bast

In [29]:
#https://stackoverflow.com/questions/11865685/handling-a-timeout-error-in-python-sockets
import errno
import os
import signal
import functools

class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wrapper

    return decorator

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [30]:
@timeout(60)
def scrape_product(start, end, product):
    df_tweets = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','product'])
    df_collect = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
        f'{product} geocode:53.81653,-3.03033,400km since:{start} until:{end}').get_items(), 2000))
    df_collect['product'] = product
    df_tweets = pd.concat([df_tweets, df_collect])#, df_BBC, df_Amazon
    return df_tweets

start_date = date(2015, 12, 31)
end_date = date(2022, 8, 2)
dates = pd.date_range(start = start_date, end = end_date,freq='D').strftime('%Y-%m-%d').tolist()

count=1
df_collected = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','product'])
df_new = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','product'])
for single_date in tqdm(daterange(start_date, end_date)):
    next_date = single_date + datetime.timedelta(days=1)
    df_new = df_new.drop(df_new.index,inplace=True)
    df_collected = df_collected.drop(df_collected.index,inplace=True)
    for product in tqdm(top_100_list):
        try:
            #print(f'doing, single {single_date} next {next_date} product {product}')
            df_new = scrape_product(single_date, next_date, product)
        except TimeoutError:
            print(f'timeout, single {single_date} next {next_date} product {product}')
            continue
        df_collected = pd.concat([df_collected, df_new])
    df_collected.to_csv(f'tweets/tweets{single_date}-product.csv', index=False)




0it [00:00, ?it/s]
100%|██████████| 99/99 [03:41<00:00,  2.24s/it]
1it [03:41, 221.66s/it]
100%|██████████| 99/99 [03:40<00:00,  2.23s/it]
2it [07:22, 221.28s/it]
100%|██████████| 99/99 [03:45<00:00,  2.28s/it]
3it [11:08, 223.30s/it]
100%|██████████| 99/99 [03:43<00:00,  2.26s/it]
4it [14:52, 223.61s/it]
100%|██████████| 99/99 [03:53<00:00,  2.35s/it]
5it [18:45, 227.08s/it]
100%|██████████| 99/99 [03:50<00:00,  2.33s/it]
6it [22:36, 228.43s/it]
100%|██████████| 99/99 [03:46<00:00,  2.29s/it]
7it [26:23, 227.95s/it]
100%|██████████| 99/99 [03:44<00:00,  2.27s/it]
8it [30:08, 226.90s/it]
100%|██████████| 99/99 [03:52<00:00,  2.35s/it]
9it [34:01, 228.80s/it]
100%|██████████| 99/99 [03:38<00:00,  2.21s/it]
10it [37:40, 225.71s/it]
100%|██████████| 99/99 [03:41<00:00,  2.24s/it]
11it [41:21, 224.41s/it]
100%|██████████| 99/99 [03:39<00:00,  2.22s/it]
12it [45:01, 223.03s/it]
100%|██████████| 99/99 [03:43<00:00,  2.26s/it]
13it [48:45, 223.33s/it]
100%|██████████| 99/99 [03:43<00:00,  2.2

In [ ]:
'''@timeout(60)
def scrape_account(start, end, account):
    df_tweets = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','account'])
    df_collect = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
        f'from:{account} since:{start} until:{end}').get_items(), 100000)) #geocode:53.81653,-3.03033,320km
    df_collect['account'] = account
    df_tweets = pd.concat([df_tweets, df_collect])#, df_BBC, df_Amazon
    return df_tweets


start_date = date(2021, 12, 7)
end_date = date(2022, 8, 2)
dates = pd.date_range(start = start_date, end = end_date,freq='D').strftime('%Y-%m-%d').tolist()

count=1
df_collected = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','account'])
df_new = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','product'])
accounts = ['BBCBreaking', 'BBCNews', '10DowningStreet', 'RoyalFamily', 'MayorofLondon', 'Tfl',
'AmazonUK', 'British_Airways'	, 'guardiannews', 'Telegraph', 'TheSun', 'thetimes', 'DailyMirror',
'Daily_Express', 'ft', 'DailyMailUK', 'standardnews', 'EveningStandard', 'TheScotsman', 'MartinSLewis', 'MoneySavingExp', 'WhichUk',
'Burberry', 'NandosUK', 'PenguinUKBooks', 'TopShop', 'ASOS', 'StellaMcCartney', 'NETAPORTER', 'unitedbus', 'BondandMayfairX',
]
for single_date in tqdm(daterange(start_date, end_date)):
    next_date = single_date + datetime.timedelta(days=1)
    df_new = df_new.drop(df_new.index,inplace=True)
    df_collected = df_collected.drop(df_collected.index,inplace=True)
    for account in tqdm(accounts):
        try:
            #print(f'doing, single {single_date} next {next_date} product {product}')
            df_new = scrape_account(single_date, next_date, account)
        except TimeoutError:
            print(f'timeout, single {single_date} next {next_date} account {account}')
            continue
        df_collected = pd.concat([df_collected, df_new])
    df_collected.to_csv(f'tweets/tweets{single_date}-accounts.csv', index=False)'''


In [ ]:
'''@timeout(60)
def scrape_any(start, end):
    df_tweets = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','any'])
    df_collect = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
        f'geocode:53.81653,-3.03033,320km since:{start} until:{end}').get_items(), 1000)) #geocode:53.81653,-3.03033,320km
    df_collect['any'] = 'any'
    df_tweets = pd.concat([df_tweets, df_collect])#, df_BBC, df_Amazon
    return df_tweets

start_date = date(2019, 7, 8)
end_date = date(2022, 8, 2)
dates = pd.date_range(start = start_date, end = end_date,freq='D').strftime('%Y-%m-%d').tolist()

count=1
df_collected = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','account'])
df_new = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','product'])

for single_date in tqdm(daterange(start_date, end_date)):
    next_date = single_date + datetime.timedelta(days=1)
    df_new = df_new.drop(df_new.index,inplace=True)
    df_collected = df_collected.drop(df_collected.index,inplace=True)
    df_new = scrape_any(single_date, next_date)
    df_collected = pd.concat([df_collected, df_new])
    df_collected.to_csv(f'tweets/tweets{single_date}-any.csv', index=False)'''

In [ ]:
'''#@timeout(60)
def scrape_any(start, end):
    df_tweets = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','uk'])
    df_collect = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
        f'uk geocode:53.81653,-3.03033,320km since:{start} until:{end}').get_items(), 1000)) #geocode:53.81653,-3.03033,320km
    df_collect['uk'] = 'uk'
    df_tweets = pd.concat([df_tweets, df_collect])#, df_BBC, df_Amazon
    return df_tweets

start_date = date(2016, 9, 5)
end_date = date(2022, 8, 1)
dates = pd.date_range(start = start_date, end = end_date,freq='D').strftime('%Y-%m-%d').tolist()

count=1
df_collected = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','uk'])
df_new = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','uk'])

for single_date in tqdm(daterange(start_date, end_date)):
    next_date = single_date + datetime.timedelta(days=1)
    df_new = df_new.drop(df_new.index,inplace=True)
    df_collected = df_collected.drop(df_collected.index,inplace=True)
    df_new = scrape_any(single_date, next_date)
    df_collected = pd.concat([df_collected, df_new])
    df_collected.to_csv(f'tweets/tweets{single_date}-uk.csv', index=False)''''